**Author name:** Thanh Chung Nguyen

In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
from datetime import datetime
import csv

In [2]:



def read_csv(fileName):
    '''Read the CSV file clickstream.csv'''
    '''you can use csv.DictReader'''
    list = []
    with open(fileName, 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            list.append(row)
    return list

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
#         print(f'Message published successfully. data:' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    
    topic_bu = 'bureau'
    topic_cus = "customer"
    bu_df = read_csv('bureau.csv')
    
    cus_df = read_csv("customer.csv")
    
    print('Publishing records..')
    producer = connect_kafka_producer()

#     len_row_bu = list(range(len(bu_df)))
#     len_row_cus = list(range(len(cus_df)))
    
    count = 1
    
    start_index = 0
    while True:
        num_rows = random.randint(10,31)
        
        dt = datetime.now()
        ts = datetime.timestamp(dt)
        alist = []
        
        to_send_bu = cus_df[start_index:num_rows+start_index]
        
        start_index = num_rows + start_index
        
        
        customer = []
        for i in to_send_bu: 
            i['ts'] = int(ts)
            
        customer += to_send_bu
        
        id_map = [i['ID'] for i in customer]
        
        bureau = [] 
        for i in bu_df:
            if i['ID'] in id_map:
                i['ts'] = int(ts)
                bureau.append(i)
            

        # publish message of producer
        publish_message(producer, topic_bu, bureau)
        publish_message(producer, topic_cus, customer)
        
        if start_index >= len(cus_df):
            start_index = 0
        sleep(5)
        
        
    
    

Publishing records..


KeyboardInterrupt: 